In [ ]:
!pip install pycaret

In [ ]:
!pip install missingno

In [23]:
# --- Importing Libraries ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as mso
import seaborn as sns
import warnings
import pycaret
from datetime import datetime

In [24]:
from pycaret.classification import *

In [25]:
# --- Libraries Settings ---
warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 100

In [45]:
# Load Application Record
application_df = pd.read_csv('data_set/application_record.csv')

# Load Credit Record
credit_df = pd.read_csv('data_set/credit_record.csv')

    Drop Dubplicate 

In [46]:
print(f"Rows before removing duplicates: {len(application_df)}")
application_df = application_df.drop_duplicates(keep='first')
print(f"Rows after removing duplicates: {len(application_df)}")

Rows before removing duplicates: 438557
Rows after removing duplicates: 438557


	3.	Process Application Record:
	•	Sort and Remove Duplicates:

In [27]:
print(application_df['DAYS_BIRTH'].head())

0   -12005
1   -12005
2   -21474
3   -19110
4   -19110
Name: DAYS_BIRTH, dtype: int64


In [28]:
application_df = application_df.sort_values(by='ID').drop_duplicates(subset='ID')

•	Replace ‘Y’/‘N’ with 1/0:

In [29]:
application_df['FLAG_OWN_CAR'] = application_df['FLAG_OWN_CAR'].map({'Y': 1, 'N': 0})
application_df['FLAG_OWN_REALTY'] = application_df['FLAG_OWN_REALTY'].map({'Y': 1, 'N': 0})

	•	Add ‘Current_Date’ Column:

	•	Calculate Age and Years Employed:

In [30]:
print(application_df['DAYS_BIRTH'].head())

0   -12005
1   -12005
2   -21474
3   -19110
4   -19110
Name: DAYS_BIRTH, dtype: int64


In [31]:
print(application_df['DAYS_EMPLOYED'].describe())

count    438510.000000
mean      60566.221347
std      138770.058166
min      -17531.000000
25%       -3103.000000
50%       -1467.000000
75%        -371.000000
max      365243.000000
Name: DAYS_EMPLOYED, dtype: float64


3. Process application_record.csv

(a) จัดการค่าที่ผิดปกติใน DAYS_EMPLOYED:

แทนค่าที่ผิดปกติ (365243) ด้วย NaN:
    1.  ค่า 365243 คืออะไร?
	    •	ในคอลัมน์ DAYS_EMPLOYED ค่า 365243 เป็นตัวแทนของ “ข้อมูลไม่ถูกต้อง” หรือ “ไม่มีข้อมูล” (เช่น คนที่ไม่มีงานทำ หรือไม่ได้ระบุวันเริ่มทำงาน)
	    •	ค่านี้ไม่สมเหตุสมผล (เพราะ 365243 วัน = 1,000 ปี!) ดังนั้นจึงต้องจัดการกับมันก่อนทำการวิเคราะห์
	2.	ทำอะไรในคำสั่งนี้?
	    •	.replace({365243: np.nan}) ใช้แทนค่าทั้งหมดที่เป็น 365243 ในคอลัมน์ DAYS_EMPLOYED ด้วย np.nan
	    •	np.nan ใน Python หมายถึง “ไม่มีข้อมูล” (missing value)
	3.	ทำไมต้องทำแบบนี้?
	    •	ถ้าปล่อยให้ค่า 365243 อยู่ จะทำให้คำนวณผิด เช่น:
	    •	ค่าเฉลี่ยของการทำงาน (mean) จะสูงเกินจริง
	    •	การคำนวณปีที่ทำงาน (-DAYS_EMPLOYED // 365) จะได้ค่าที่ไม่สมเหตุสมผล เช่น -1000 ปี
        •	เมื่อเปลี่ยนเป็น np.nan โปรแกรมจะมองข้ามค่าที่ไม่มีข้อมูลเวลาคำนวณ เช่น ค่าเฉลี่ย หรือยอดรวม

ก่อนแทนที่:
DAYS_EMPLOYED
-3000
365243
-5000
365243

หลัง:
DAYS_EMPLOYED
-3000
NaN
-5000
NaN


สรุป:

คำสั่งนี้ช่วยจัดการข้อมูลที่ผิดปกติหรือไม่มีข้อมูลให้กลายเป็น NaN เพื่อป้องกันไม่ให้คำนวณผิด และให้ผลลัพธ์แม่นยำขึ้น เช่น การหาค่าเฉลี่ยหรือการวิเคราะห์ข้อมูลในภายหลัง

In [32]:
# Replace placeholder value in DAYS_EMPLOYED
application_df['DAYS_EMPLOYED'] = application_df['DAYS_EMPLOYED'].replace({365243: np.nan})

(b) คำนวณอายุ (AGE) และจำนวนปีทำงาน (YEARS_EMPLOYED):

In [33]:
# คำนวณอายุ
application_df['AGE'] = (-application_df['DAYS_BIRTH']) // 365

# คำนวณจำนวนปีที่ทำงาน (เฉพาะค่าที่ไม่ใช่ NaN)
application_df['YEARS_EMPLOYED'] = (-application_df['DAYS_EMPLOYED']) // 365

In [34]:
print(application_df[['AGE', 'YEARS_EMPLOYED']].describe())

                 AGE  YEARS_EMPLOYED
count  438510.000000    363186.00000
mean       43.326604         6.68430
std        11.467615         6.57934
min        20.000000         0.00000
25%        34.000000         2.00000
50%        42.000000         5.00000
75%        53.000000         9.00000
max        69.000000        48.00000


4. Process credit_record.csv

    (a) จัดการสถานะ (STATUS):
    แปลง STATUS ให้เป็น “Good Debt” หรือ “Bad Debt”:

In [35]:
# Define good and bad debt statuses
good_debt_statuses = ['0', 'C', 'X']
bad_debt_statuses = ['1', '2', '3', '4', '5']

# สร้างคอลัมน์ใหม่แยก Good และ Bad Debt
credit_df['Good_Debt'] = credit_df['STATUS'].apply(lambda x: 1 if x in good_debt_statuses else 0)
credit_df['Bad_Debt'] = credit_df['STATUS'].apply(lambda x: 1 if x in bad_debt_statuses else 0)

(b) รวมข้อมูลสถานะโดย ID:

คำนวณจำนวนเดือนที่เป็น Good Debt และ Bad Debt สำหรับแต่ละคน:

In [36]:
debt_summary = credit_df.groupby('ID').agg(
    Good_Debt_Count=('Good_Debt', 'sum'),
    Bad_Debt_Count=('Bad_Debt', 'sum')
).reset_index()

5. รวมข้อมูลจากทั้งสองไฟล์

รวมข้อมูลจาก application_df และ credit_df โดยใช้ ID เป็นตัวเชื่อม:

In [37]:
merged_df = pd.merge(application_df, debt_summary, on='ID', how='inner')

6. จัดหมวดหมู่สถานะโดยรวม

สร้างคอลัมน์ STATUS เพื่อจัดหมวดหมู่คนที่มีหนี้ดีหรือหนี้เสีย:

In [38]:
merged_df['STATUS'] = merged_df.apply(
    lambda row: 1 if row['Good_Debt_Count'] > row['Bad_Debt_Count'] else 0,
    axis=1
)

7. Export ข้อมูลที่จัดการแล้ว

บันทึกข้อมูลที่ผ่านการประมวลผลลงในไฟล์ CSV:

In [39]:
merged_df.to_csv('cleaned_credit_card_data.csv', index=False)

In [43]:
# Check for duplicates
application_df = pd.read_csv('cleaned_credit_card_data.csv')
duplicates = application_df[application_df.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [ID, CODE_GENDER, FLAG_OWN_CAR, FLAG_OWN_REALTY, CNT_CHILDREN, AMT_INCOME_TOTAL, NAME_INCOME_TYPE, NAME_EDUCATION_TYPE, NAME_FAMILY_STATUS, NAME_HOUSING_TYPE, DAYS_BIRTH, DAYS_EMPLOYED, FLAG_MOBIL, FLAG_WORK_PHONE, FLAG_PHONE, FLAG_EMAIL, OCCUPATION_TYPE, CNT_FAM_MEMBERS, AGE, YEARS_EMPLOYED, Good_Debt_Count, Bad_Debt_Count, STATUS]
Index: []

[0 rows x 23 columns]


In [44]:
print(f"Rows before removing duplicates: {len(application_df)}")
application_df = application_df.drop_duplicates(keep='first')
print(f"Rows after removing duplicates: {len(application_df)}")

Rows before removing duplicates: 36457
Rows after removing duplicates: 36457
